# Environmental day Web Scrapping - Data Prepare

In [1]:
import time
import json
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from lxml import etree
import xmltodict
import urllib3
from selenium.common.exceptions import InvalidSessionIdException
from selenium.common.exceptions import WebDriverException

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [21]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.maximize_window()
time.sleep(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/fj/3rtd_zxj3fd8fx2vq1knfgdc0000gn/T/ipykernel_91817/1983766380.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


## Scrapping from UN NEWS

In [2]:
MAIN_URL = "https://news.un.org/"

In [3]:
articles_url = []

In [87]:
for i in range(1,93):
    browser.get(f"https://news.un.org/en/news/topic/climate-change?page={i}") 
    get_page = bs(browser.page_source)
    for stories in get_page('div',{'class':'view-content'})[0]('h1',{'class':'story-title'}):
        articles_url.append(stories('a')[0]['href'])
    print(f"---Finished page{i}---")
    time.sleep(10)

---Finished page1---
---Finished page2---
---Finished page3---
---Finished page4---
---Finished page5---
---Finished page6---
---Finished page7---
---Finished page8---
---Finished page9---
---Finished page10---
---Finished page11---
---Finished page12---
---Finished page13---
---Finished page14---
---Finished page15---
---Finished page16---
---Finished page17---
---Finished page18---
---Finished page19---
---Finished page20---
---Finished page21---
---Finished page22---
---Finished page23---
---Finished page24---
---Finished page25---
---Finished page26---
---Finished page27---
---Finished page28---
---Finished page29---
---Finished page30---
---Finished page31---
---Finished page32---
---Finished page33---
---Finished page34---
---Finished page35---
---Finished page36---
---Finished page37---
---Finished page38---
---Finished page39---
---Finished page40---
---Finished page41---
---Finished page42---
---Finished page43---
---Finished page44---
---Finished page45---
---Finished page46-

In [91]:
with open("","w") as file:
    json.dump(articles_url, file)
#DO NOT RUN THIS CELL UNLESS YOU KNOW WHAT YOU ARE DOING

In [5]:
with open("articles_url.json","r") as file:
    data = json.load(file)
len(data)

929

In [79]:
articles = []
checkpoint = 50
count = 0

In [82]:
for url in data[170:]:
    browser.get(MAIN_URL + url)
    content_page = bs(browser.page_source)
    article = {}
    try:
        article["title"] = content_page('div',{'class':'row before-content'})[0]('h1',{'class','story-title'})[0].get_text().strip()
    except IndexError:
        try:
            article["title"] = content_page('div',{'class':'quote-text'})[0]('h2',{'class','story-title'})[0].get_text().strip()
        except IndexError:
            article["title"] = content_page('h1',{'class':'story-title'})[0].get_text().strip()
    try:
        text_content = content_page('div',{'class','field-name-field-news-story'})[0]('p')
    except IndexError:
        continue 
    content = ""
    for i in text_content:
        content += (i.get_text())
    article["text"] = content.strip()
    articles.append(article)
    count += 1
    if count % checkpoint == 0:
        with open(f"articles_content_{count}.json","w",encoding = "utf8") as file:
            json.dump(articles,file,ensure_ascii=False)
        print(f"Checkpoint {count} passed----")
        time.sleep(10)
    if count % 100 == 0:
        time.sleep(20)
    

Checkpoint 200 passed----
Checkpoint 250 passed----
Checkpoint 300 passed----
Checkpoint 350 passed----
Checkpoint 400 passed----
Checkpoint 450 passed----
Checkpoint 500 passed----
Checkpoint 550 passed----
Checkpoint 600 passed----
Checkpoint 650 passed----
Checkpoint 700 passed----
Checkpoint 750 passed----
Checkpoint 800 passed----
Checkpoint 850 passed----
Checkpoint 900 passed----


In [83]:
with open(f"articles_content_{count}.json","w",encoding = "utf8") as file:
    json.dump(articles,file, ensure_ascii=False)
#DO NOT RUN THIS CELL UNLESS YOU KNOW WHAT YOU ARE DOING

In [84]:
len(articles)

918

## Scrapping from ScienceDaily

In [3]:
MAIN_URL = "https://www.sciencedaily.com/newsfeeds.htm"

In [112]:
for i in s('div',{'class','table-responsive'})[3]('table',{'class','table'})[0]('td'):
    for j in i('li'):
        try:
            scienceDaily.append(j('a')[0]['href'])
        except IndexError:
            continue

In [166]:
for link in scienceDaily:
    browser.get("https://www.sciencedaily.com" + link)
    s = bs(browser.page_source)
    content = s.find('body').get_text()
    root = xmltodict.parse(content)
    topic_url.append(root['rss']['channel']['link'])
    for item in root['rss']['channel']['item']:
        scienceDaily_url.append(item['link'])

In [167]:
len(topic_url), len(scienceDaily_url)

(274, 8220)

In [168]:
with open("scienceDaily_url.json","w") as file:
    json.dump(scienceDaily_url, file)

In [169]:
with open("topic_url.json","w") as file:
    json.dump(topic_url,file)

In [5]:
with open("scienceDaily_url.json","r") as file:
    scienceDaily_url = json.load(file)
len(scienceDaily_url)

8220

In [13]:
scienceDaily_articles = []
count = 1074
checkpoint = 100

In [22]:
for url in scienceDaily_url[5216:]:
    try:
        browser.get(url)
    except (InvalidSessionIdException, WebDriverException) as err :
        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.maximize_window()
        time.sleep(1)
        browser.get(url)
    
    try:
        s = bs(browser.page_source)
    except WebDriverException:
        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.maximize_window()
        time.sleep(1)
        browser.get(url)
        s = bs(browser.page_source)
    article = {}
    article['title'] = s.find(id='headline').get_text().strip()
    content = s.find(id='first').get_text().strip()
    for string in s.find(id='text')('p'):
        content += string.get_text().strip()
    article['content'] = content
    count += 1
    scienceDaily_articles.append(article)
    if count % checkpoint == 0:
        with open(f"scienceDaily_article_start=1074_end={1074+count}.json","w",encoding="utf8") as file:
            json.dump(scienceDaily_articles,file,ensure_ascii=False)
        print(f"Checkpoint {count} passed -----")
        time.sleep(10)
    if count % 100 == 0:
        time.sleep(20)

Checkpoint 5300 passed -----
Checkpoint 5400 passed -----
Checkpoint 5500 passed -----
Checkpoint 5600 passed -----



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/fj/3rtd_zxj3fd8fx2vq1knfgdc0000gn/T/ipykernel_91817/3373506621.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


Checkpoint 5700 passed -----
Checkpoint 5800 passed -----
Checkpoint 5900 passed -----
Checkpoint 6000 passed -----



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Checkpoint 6100 passed -----
Checkpoint 6200 passed -----
Checkpoint 6300 passed -----
Checkpoint 6400 passed -----



...

Checkpoint 7000 passed -----
Checkpoint 7100 passed -----
Checkpoint 7200 passed -----
Checkpoint 7300 passed -----
Checkpoint 7400 passed -----



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Checkpoint 7500 passed -----
Checkpoint 7600 passed -----
Checkpoint 7700 passed -----
Checkpoint 7800 passed -----
Checkpoint 7900 passed -----
Checkpoint 8000 passed -----



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Checkpoint 8100 passed -----
Checkpoint 8200 passed -----


In [23]:
with open(f"scienceDaily_article_start=1074_end={1074+count}.json","w",encoding="utf8") as file:
    json.dump(scienceDaily_articles,file,ensure_ascii=False)

In [27]:
count

8219

In [25]:
with open(f"scienceDaily_article_start=1074_end=9293.json","r") as file:
    test = json.load(file)
len(test) + 1074

8219

In [219]:
with open(f"scienceDaily_article_{count}.json","w",encoding="utf8") as file:
    json.dump(scienceDaily_articles,file,ensure_ascii=False)

# Scrapping from RealWire 

In [52]:
browser.get("https://www.realwire.com/rss/feeds.asp?cat=Environmental")
s = bs(browser.page_source) 

In [97]:
RealWire_link = []

In [98]:
for i,string in enumerate(s.find(id='feedTable')('tr')[0]('td')[1]('a')):
    if i % 2 == 1:
        RealWire_link.append(string['href'][2:])

# Data Cleaning

In [26]:
data = []
for i in range(2,15):
    with open(f"Huffington Post/backup{i}.json", "r") as file:
        data += json.load(file)
len(data)

16748

In [31]:
import pandas as pd
df = pd.DataFrame(data)
df = df.drop_duplicates(subset = ['title'])
len(df)

14354

In [32]:
import os
data = []
fList = os.listdir("other_article")
for f in fList[0:2]:
    print(f)
    with open("other_article/"+f, "r") as file:
        data += json.load(file)
len(data)

scienceNews_article_3568.json
articles_content_918.json


4486

In [33]:
df2 = pd.DataFrame(data)
df2 = df2.drop_duplicates(subset = ['title'])
len(df2)


4405

In [34]:
data = []
for f in fList[2:5]:
    print(f)
    with open("other_article/"+f, "r") as file:
        data += json.load(file)
len(data)

scienceDaily_article_1073.json
scienceDaily_article_start=1074_end=9293.json


8218

In [35]:
df3 = pd.DataFrame(data)
df3.columns = ['title','text']
df3 = df3.drop_duplicates()
len(df3)

3679

In [25]:
total = pd.concat([df,df2,df3])
len(total)

22438

In [36]:
total.to_csv("total_data.csv")